# Data types á la carte

## 2. Fixing the expression problem

In [1]:
data Expr f = In (f (Expr f)) -- same thing as Fix in recursion schemes

-- things they don't explain in this paper: the "magic" in the above definition.
-- how it solves the arbitrary nested type problem

data Val e = Val Int
data Add e = Add e e

{-# LANGUAGE TypeOperators #-}

data (f :+: g) e = Inl (f e) | Inr (g e)

-- `Expr (Val :+: Add)`: an expression built from subexpressions of type `Val` or `Add`

addExample :: Expr (Val :+: Add)
addExample = In (Inr (Add (In (Inl (Val 118))) (In (Inl (Val 1219)))))

-- fixes the expression problem b/c you can add new types and new functions easily?
-- how to eval an expression like this?

## 3. Evaluation

In [2]:
instance Functor Val where
    fmap f (Val x) = Val x
    
instance Functor Add where
    fmap f (Add l r) = Add (f l) (f r) 
    
instance (Functor f, Functor g) => Functor (f :+: g) where
    fmap f (Inl l) = Inl (fmap f l)
    fmap f (Inr r) = Inr (fmap f r)
    
foldExpr :: Functor f => (f a -> a) -> Expr f -> a -- same as cata in recursion schemes, I think
foldExpr f (In t) = f (fmap (foldExpr f) t)

* `f a -> a` is an algebra: something that does one step of the recursion using the results of the recursive calls
* Idea: define and assemble algebras from individual cases (e.g. `Val` or `Add`) in order to define the algebra for the coproduct (e.g. `Val :+: Add`). We can use the type class system for this and teach it how to compose algebras.

In [3]:
-- A separate class for the algebra we aim to define:
class Functor f => Eval f where
    evalAlgebra :: f Int -> Int
    
instance Eval Val where
    evalAlgebra (Val x) = x
    
instance Eval Add where
    evalAlgebra (Add x y) = x + y

instance (Eval f, Eval g) => Eval (f :+: g) where
    evalAlgebra (Inl l) = evalAlgebra l
    evalAlgebra (Inr r) = evalAlgebra r
    
eval :: Eval f => Expr f -> Int
eval expr = foldExpr evalAlgebra expr

eval addExample

1337

## 4. Automating injections

A first attempt:

In [4]:
val :: Int -> Expr Val
val x = In (Val x)

infixl 6 <+>

(<+>) :: Expr Add -> Expr Add -> Expr Add
x <+> y = In (Add x y)

val 1 <+> val 2 -- d'oh

<interactive>:1:1: error:
    • Couldn't match type ‘Val’ with ‘Add’
      Expected type: Expr Add
        Actual type: Expr Val
    • In the first argument of ‘(<+>)’, namely ‘val 1’
      In the expression: val 1 <+> val 2
      In an equation for ‘it’: it = val 1 <+> val 2
<interactive>:1:11: error:
    • Couldn't match type ‘Val’ with ‘Add’
      Expected type: Expr Add
        Actual type: Expr Val
    • In the second argument of ‘(<+>)’, namely ‘val 2’
      In the expression: val 1 <+> val 2
      In an equation for ‘it’: it = val 1 <+> val 2

Idea: make signatures less rigid

```haskell
(⊕) :: (Add :≺: f ) ⇒ Expr f → Expr f → Expr f
val :: (Val :≺: f ) ⇒ Int → Expr f
```

> The constraint `sub :≺: sup` should only be satisfied if there is some injection from `sub a` to `sup a`.

Is `:≺:` a natural transformation?

In [5]:
{-# LANGUAGE MultiParamTypeClasses #-}

-- Idea: rather than writing `Inr` or `Inl` by hand,the 
-- injections can be inferred using instances of this class:
class (Functor sub, Functor sup) => sub :<: sup where
    inj :: sub a -> sup a
    
{-# LANGUAGE FlexibleInstances #-}

-- "Constructors" that use the types:
    
-- reflexivity: something can be injected to itself (when is this one useful?)
instance Functor f => f :<: f where
    inj = id

-- "if f is true then f or g is true". 
-- Inject anything into any coproduct that contains it explicitly
instance (Functor f , Functor g) => f :<: (f :+: g) where
    inj = Inl

-- Inject anything into any coproduct that contains something it can be injected into
instance (Functor f , Functor g, Functor h, f :<: g) => f :<: (h :+: g) where
    inj = Inr . inj

In [6]:
inject :: (g :<: f) => g (Expr f) -> Expr f
inject = In . inj